[View in Colaboratory](https://colab.research.google.com/github/taataam/UHOFWorkshop/blob/master/workshop3/ACM_Discretization.ipynb)

# Governing Equations

The Incompressible Navier-Stokes Equations (INSE) is as follows:
\begin{gather}
\partial_t \rho + \rho \nabla . \mathbf{u} =0\\
\partial_t \mathbf{u} + \mathbf{u} . \nabla \mathbf{u} = -\frac{1}{\rho} \nabla p + \nu \nabla^2 \mathbf{u} + \mathbf{g}
\end{gather}

Since there is no separate equation for pressure we should decouple the equations in some way. Artificial Compressibility Method (ACM) offers a technique for decoupling pressure and velocity by assuming small compressibility for the fluid and isothermal condition for the flow. These assumptions suggest that density becomes only a function of pressure. Therefore, we have:
\begin{gather}
\rho = \rho(p) \therefore \partial_t \rho = \frac{\partial \rho}{\partial p} \frac{\partial p}{\partial t} = \frac{1}{c^2} \frac{\partial p}{\partial t}
\end{gather}

where $c$ is artificial sound speed and has the same dimesion as velocity. This assumption means that there is a time lag between the flow disturbance and its effect on the pressure field. The INSE can be written as follows:
\begin{gather}
\partial_t P + c^2 \nabla . \mathbf{u} =0\\
\partial_t \mathbf{u} + \mathbf{u} . \nabla \mathbf{u} = -\nabla P + \nu \nabla^2 \mathbf{u} + \mathbf{g}
\end{gather}
where $P = \dfrac{p}{\rho}$.

These assumptions significantly facilitate efficient solution of the equations at the cost of limiting scope of applicability of the method; incompressible steady state problems.

Some remarks about ACM:
- $t$ is pseudo-time step and depends on $c$. Therefore, if an explicit discretization method is used time step should have a small value.
- Since the solution does not invlove any elliptic PDE which is non-local, the method can be parallelized efficiently.
- Although applicability of the method is very limited, it provides a simple yet informative framework for pedagogical purposes.

# Solution algorithm:

Solution procedure for inner grid points can be formulated as follows assuming that superscript $n$ denotes the unknown variable:
1. The **momentum equation in $x$ direction** is as follows:
\begin{equation}
\partial_t u + \partial_x(u u) + \partial_y(u v) = - \partial_x P + \nu \left(\partial_{xx} u + \partial_{yy} u\right)
\end{equation}

We can discretisize the equation using a central differencing schemes:
\begin{align}
\frac{u^n_{i,j} - u_{i,j}}{\Delta t}&+
\frac{\left(\frac{u_{i+1,j} + u_{i,j}}{2}\right)^2 - \left(\frac{u_{i,j} + u_{i-1,j}}{2}\right)^2}{\Delta x}\\
&+
\frac{\left(\frac{u_{i,j+1} + u_{i,j}}{2}\right)  \left(\frac{v_{i+1,j} + v_{i,j}}{2}\right) - \left(\frac{u_{i,j} + u_{i,j-1}}{2}\right)  \left(\frac{v_{i+1,j-1} + v_{i,j-1}}{2}\right)}{\Delta y}\\
&=\\
&- \frac{P_{i+1,j} - P_{i,j}}{\Delta x}\\
&+\nu \left( \frac{u_{i+1,j} - 2u_{i,j} +u_{i-1,j} }{\Delta x^2} +  \frac{u_{i,j+1} - 2u_{i,j} +u_{i,j-1} }{\Delta y^2}\right)
\end{align}

2. The **momentum equation in $y$ direction** is as follows:
\begin{equation}
\partial_t v + \partial_x(u v) + \partial_y(v v) = - \partial_y P + \nu \left(\partial_{xx} v + \partial_{yy} v\right)
\end{equation}

We can discretisize the equation using a central differencing schemes:

\begin{align}
\frac{v^n_{i,j} - v_{i,j}}{\Delta t}&+
\frac{\left(\frac{u_{i,j+1} + u_{i,j}}{2}\right)  \left(\frac{v_{i+1,j} + v_{i,j}}{2}\right) - \left(\frac{u_{i-1,j+1} + u_{i-1,j}}{2}\right)  \left(\frac{v_{i,j} + v_{i-1,j}}{2}\right)}{\Delta x}\\
&+
\frac{\left(\frac{v_{i,j+1} + v_{i,j}}{2}\right)^2 - \left(\frac{v_{i,j} + v_{i,j-1}}{2}\right)^2}{\Delta y}\\
&=\\
&- \frac{P_{i,j+1} - P_{i,j}}{\Delta y}\\
&+\nu \left( \frac{v_{i+1,j} - 2v_{i,j} +v_{i-1,j} }{\Delta x^2} +  \frac{v_{i,j+1} - 2v_{i,j} +v_{i,j-1} }{\Delta y^2}\right)
\end{align}

3. The **continuity equation** is as follows:
\begin{equation}
\partial_t P + c^2 \left(\partial_x u+ \partial_y v \right) =  0
\end{equation}

We can discretisize the equation using a central differencing schemes:

\begin{align}
 \frac{P^n_{i,j} - P_{i,j}}{\Delta t} + c^2 \left( \frac{u^n_{i,j} - u^n_{i-1,j} }{\Delta x} +  \frac{v^n_{i,j} - v^n_{i,j-1} }{\Delta y}\right) = 0
\end{align}

# Boundary Conditions:

The two common boundary conditions can be formulated as follows:

- Dirichlet: $\phi_g = 2 \phi_b - \phi_i$
- Neumann: $\phi_g = \phi_i - \left( \Delta n \frac{\partial \phi}{\partial n} \right)_b$

where $g$, $b$ and $i$ denote a ghost, boundary and inner node, respectively. Also $\phi$ is a quantity of interest ($u$, $v$ or $P$) and $n$ represents a direction normal to the boundary cell face.

In the lid-driven cavity problem velocity boundary condition is Dirichlet on all the boundaries and pressure is Neumann.

# Convergence criteria:
The following criteria are used for measuring convergence of the solution:
- $E_u = \sqrt{(\Delta t \Delta x \Delta  y) \sum_{i,j}(u^{n+1}_{i,j} - u^{n}_{i,j})^2}$
- $E_v = \sqrt{(\Delta t \Delta x \Delta  y) \sum_{i,j}(v^{n+1}_{i,j} - v^{n}_{i,j})^2}$
- $E_p = \sqrt{\frac{\Delta t \Delta x \Delta  y }{c^2} \sum_{i,j}(P^{n+1}_{i,j} - P^{n}_{i,j})^2}$
- $E_{\nabla} = (\Delta t \Delta x \Delta  y) \nabla . \mathbf{u}$
- $E_{tot} = \max\{E_u, E_v, E_p, E_{\nabla}) < \varepsilon$

In the lid-driven cacity problem, $\varepsilon$ which is tolerence, is set to $10^{-8}$.